In [15]:
import pandas as pd
import pandas_ta as ta
import talib
import requests
import time
import datetime
import json
import yfinance as yf
import mplfinance as mpf
from io import StringIO
#from google.colab import data_table
#data_table.enable_dataframe_formatter()

In [ ]:
def get_stock_list():
  res = requests.get("https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y")
  df = pd.read_html(res.text)[0]        # Parse the Source Code into a Pandas DataFrame
  df = df.drop([0,1,4,5,8,9],axis = 1)  # Drop Useless Columns
  df.columns = df.iloc[0]               # Replace DataFrame Columns Title
  df = df.iloc[1:]
  return df

In [ ]:
def plot_stcok_k_chart(CLIENT_ID, stock="0050" , date_from='2020-01-01' ):
  """
  進行個股K線繪製，回傳至於雲端圖床的連結。將顯示包含5MA、20MA及量價關係，預設為'2020-01-01'迄今收盤價。
  :stock :個股代碼(字串)，預設0050。
  :date_from :起始日(字串)，格式為%Y-%m-%d，預設自2020-01-01起。
  """
  stock = str(stock)+".tw"
  # df = web.DataReader(stock, 'yahoo', date_from) # 已知有未修復的錯誤
  df = yf.download(stock, date_from)
  mpf.plot(df,type='candle',mav=(5,20),volume=True,title=stock.upper() ,savefig='testsave.png')
  PATH = "testsave.png"
  im = pyimgur.Imgur(CLIENT_ID)
  uploaded_image = im.upload_image(PATH, title=stock+" candlestick chart")
  return uploaded_image.link

# Main get stocker data

In [ ]:
import requests
from io import StringIO
import pandas as pd
import pandas_ta as ta
import numpy as np
import datetime
import time
import sqlite3

class Stocker:
  def __init__(self, n_days, db_name):
    #self.name = name
    self.today = datetime.datetime.now()
    self.country = "TW"
    self.source = "TWSE"
    self.rank = ""
    self.data = None
    self.data_ta = None
    self.OHLCV = None
    self.n_days = n_days
    self.db_name = db_name

  def crawl_price(self, date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    #ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '})
    #                                    for i in r.text.split('\n')
    #                                    if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = pd.read_csv(StringIO(r.text.replace("=", "")),
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    ret = ret.set_index('證券代號')
    ret = ret[:'備註:'][:-1]
    ret = ret.loc[:,:'本益比']
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    ret['成交筆數'] = ret['成交筆數'].str.replace(',','')
    ret['交易日']= date.date()
    return ret


  def get_twse_stocker_date_range_date(
      self,
      startdate=datetime.datetime.now(),
      n_days=10,
      sleep_time=10,
      allow_continuous_fail_count=5,
  ):
      data = {}
      date = startdate
      fail_count = 0
      while len(data) < n_days:

          print('parsing', date)
          # 使用 crawPrice 爬資料

          try:
              if pd.to_datetime(date.date()) in self.data.index.levels[0] and self.data is not None:
                print('skip! data is already having')
                data[date.date()] = self.data.loc[[date.date()]].droplevel(level='交易日')
              else:
                # 抓資料
                data[date.date()] = self.crawl_price(date)
                print('success!')
                fail_count = 0
          except:
              # 假日爬不到
              print('fail! check the date is holiday')
              fail_count += 1
              if fail_count == allow_continuous_fail_count:
                  raise
                  break

          # 減一天
          date -= datetime.timedelta(days=1)
          time.sleep(sleep_time)
      #return data
      self.data = pd.concat(data)
  def get_data(self) -> pd.DataFrame:
    self.data = pd.concat(self.get_twse_stocker_date_range_date(n_days=self.n_days))

  def get_data_ta(self):
    self.data_ta = self.data[['開盤價', '最高價', '最低價', '收盤價', '成交股數']].copy()       
    self.data_ta.columns = ['open', 'high', 'low', 'close', 'volume']
    self.data_ta = self.data_ta.swaplevel('交易日', '證券代號').sort_index()

  def correct_data(self):
     flaot_cols = ['開盤價', '最高價', '最低價', '收盤價']
     int_cols = ['成交股數',	'成交筆數',	'成交金額']
     self.data[flaot_cols] = self.data[flaot_cols].apply(lambda x: pd.to_numeric(x, errors='coerce'))
     self.data[int_cols] = self.data[int_cols].apply(lambda x: pd.to_numeric(x, errors='coerce'))


  def save_db(self, data, db_name) -> None:
    conn = sqlite3.connect('stocker.db')  #建立資料庫
    cursor = conn.cursor()
    self.data.to_sql('Stocker', conn, if_exists='append', index=True)
  def load_db(self, db_nmae) -> None:
    conn = sqlite3.connect('stocker.db')  #建立資料庫
    cursor = conn.cursor()
    data = pd.read_sql('SELECT * FROM Stocker', conn, index_col=['交易日','證券代號'])
    self.correct_data()
    self.data=data

In [ ]:
data = get_twse_stocker_date_range_date(n_days=26)

In [2]:
import firebase_admin
from firebase_admin import firestore

# Application Default credentials are automatically created.
app = firebase_admin.initialize_app()
db = firestore.client()

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [100]:
Stocker2= Stocker(n_days=30,db_name="stocker.db")

In [101]:
Stocker2.load_db(Stocker2.db_name)

C:\Users\David\AppData\Local\Temp\ipykernel_7824\3533768478.py:101: UserWarning: Instantiating StringDtype without any arguments.Pass a StringDtype instance to silence this warning.
  data.index=data.index.set_levels(data.index.levels[1].astype(pd.StringDtype), level='證券代號')


In [99]:
del Stocker2

In [ ]:
Stocker2.save_db(Stocker2.data, Stocker2.db_name)

In [104]:
Stocker2.get_data_ta()

In [17]:
Stocker2.data.ta.indicators('supertrend')

Pandas TA - Technical Analysis Indicators - v0.4.71b0

Indicators and Utilities [154]:
    aberration, accbands, ad, adosc, adx, alligator, alma, alphatrend, amat, ao, aobv, apo, aroon, atr, atrts, bbands, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chandelier_exit, chop, cksp, cmf, cmo, coppock, crsi, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, exhc, fisher, fwma, ha, hilo, hl2, hlc3, hma, ht_trendline, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, mama, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, pivots, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, reflex, rma, roc, rsi, rsx, rvgi, rvi, rwi, short_run, sinwma, skew, slope, sma, smc, smi, smma, squeeze, squeeze_pro, ssf, ssf3, stc, stdev, stoch, stochf, stochrsi, supertrend, swma, t3, tema, thermo, tmo, tos_stdevall, trendflex

In [22]:
for k, d in Stocker2.data.items():
    print('k:',k,'d:',d)

k: level_0 d: 交易日         證券代號
2025-09-19  0050    2025-09-19
            0051    2025-09-19
            0052    2025-09-19
            0053    2025-09-19
            0055    2025-09-19
                       ...    
2025-02-19  9944          None
            9945          None
            9946          None
            9955          None
            9958          None
Name: level_0, Length: 6475503, dtype: object
k: 證券名稱 d: 交易日         證券代號
2025-09-19  0050      元大台灣50
            0051     元大中型100
            0052        富邦科技
            0053        元大電子
            0055    元大MSCI金融
                      ...   
2025-02-19  9944          新麗
            9945         潤泰新
            9946        三發地產
            9955          佳龍
            9958         世紀鋼
Name: 證券名稱, Length: 6475503, dtype: object
k: 成交股數 d: 交易日         證券代號
2025-09-19  0050    74098514
            0051       72633
            0052      394424
            0053       20490
            0055       48522
                   

In [92]:
Stocker2.data['開盤價']

交易日         證券代號
2025-09-19  0050     57.00
            0051     86.70
            0052    225.75
            0053    125.40
            0055     30.47
                     ...  
2025-02-19  9944     20.35
            9945     42.75
            9946     23.85
            9955     31.70
            9958    172.00
Name: 開盤價, Length: 6475503, dtype: object

In [59]:
stock_dfs = {}
for stock_id, df_stock in Stocker2.data.groupby(level='證券代號'):
    df_stock = df_stock.droplevel('證券代號')  # 只保留交易日當 index
    stock_dfs[stock_id] = df_stock

In [60]:
stock_dfs

{'0050':                level_0    證券名稱      成交股數   成交筆數        成交金額     開盤價     最高價  \
 交易日                                                                           
 2025-09-19  2025-09-19  元大台灣50  74098514  47867  4216285864   57.00   57.10   
 2025-09-18  2025-09-18  元大台灣50  45899106  37903  2597748047   56.50   56.95   
 2025-09-17  2025-09-17  元大台灣50  70590054  76500  3980499506   56.60   56.60   
 2025-09-16  2025-09-16  元大台灣50  65388179  48144  3705092621   56.15   56.85   
 2025-09-15  2025-09-15  元大台灣50  56031630  59230  3133893488   56.00   56.05   
 ...                ...     ...       ...    ...         ...     ...     ...   
 2025-02-25        None  元大台灣50  31389673  88376  6070255996  194.00  194.05   
 2025-02-24        None  元大台灣50  14227193  37877  2788014198  196.15  196.40   
 2025-02-21        None  元大台灣50   9097283  16607  1795160598  196.50  197.95   
 2025-02-20        None  元大台灣50  15504294  35876  3036131580  196.95  196.95   
 2025-02-19        None  元大台灣50 

In [61]:
Stocker2.data_ta = Stocker2.data[['開盤價', '最高價', '最低價', '收盤價', '成交股數']].copy()
Stocker2.data_ta.columns = ['open', 'high', 'low', 'close', 'volume']

In [89]:
Stocker2.data_ta = Stocker2.data_ta.swaplevel('交易日', '證券代號').sort_index()

In [105]:
Stocker2.data_ta

open    high    low   close    volume
證券代號 交易日                                                
0050 2025-02-19  197.65  197.90  197.0  197.30  12820440
     2025-02-20  196.95  196.95  195.1  196.55  15504294
     2025-02-21  196.50  197.95  195.9  197.95   9097283
     2025-02-24  196.15  196.40  195.5  196.20  14227193
     2025-02-25  194.00  194.05  192.6  193.65  31389673
...                 ...     ...    ...     ...       ...
9958 NaT         170.50  171.50  168.5  169.00   1303725
     NaT         169.50  171.50  169.5  170.50    878469
     NaT         169.50  171.50  169.5  170.50    878469
     NaT         172.00  174.50  168.5  168.50   1657743
     NaT         172.00  174.50  168.5  168.50   1657743

[6475503 rows x 5 columns]

In [106]:
def add_indicators(df_group):
    df_group = df_group.copy()
    df_group['SMA20'] = ta.sma(df_group['close'], length=20)
    df_group['RSI14'] = ta.rsi(df_group['close'], length=14)
    macd = ta.macd(df_group['close'])
    df_group['MACD'] = macd['MACD_12_26_9']
    df_group['MACDh'] = macd['MACDh_12_26_9']
    df_group['MACDs'] = macd['MACDs_12_26_9']
    kd = ta.stoch(df_group['high'], df_group['low'], df_group['close'])
    df_group['K'] = kd['STOCHk_14_3_3']
    df_group['D'] = kd['STOCHd_14_3_3']
    return df_group

In [107]:
add_indicators(Stocker2.data_ta)

open    high    low   close    volume  SMA20  RSI14  MACD  \
證券代號 交易日                                                                       
0050 2025-02-19  197.65  197.90  197.0  197.30  12820440    NaN    NaN   NaN   
     2025-02-20  196.95  196.95  195.1  196.55  15504294    NaN    NaN   NaN   
     2025-02-21  196.50  197.95  195.9  197.95   9097283    NaN    NaN   NaN   
     2025-02-24  196.15  196.40  195.5  196.20  14227193    NaN    NaN   NaN   
     2025-02-25  194.00  194.05  192.6  193.65  31389673    NaN    NaN   NaN   
...                 ...     ...    ...     ...       ...    ...    ...   ...   
9958 NaT         170.50  171.50  168.5  169.00   1303725    NaN    NaN   NaN   
     NaT         169.50  171.50  169.5  170.50    878469    NaN    NaN   NaN   
     NaT         169.50  171.50  169.5  170.50    878469    NaN    NaN   NaN   
     NaT         172.00  174.50  168.5  168.50   1657743    NaN    NaN   NaN   
     NaT         172.00  174.50  168.5  168.50   1657743    NaN    NaN   NaN   

                 MACDh  MACDs   K   D  
證券代號 交易日                               
0050 2025-02-19    NaN    NaN NaN NaN  
     2025-02-20    NaN    NaN NaN NaN  
     2025-02-21    NaN    NaN NaN NaN  
     2025-02-24    NaN    NaN NaN NaN  
     2025-02-25    NaN    NaN NaN NaN  
...                ...    ...  ..  ..  
9958 NaT           NaN    NaN NaN NaN  
     NaT           NaN    NaN NaN NaN  
     NaT           NaN    NaN NaN NaN  
     NaT           NaN    NaN NaN NaN  
     NaT           NaN    NaN NaN NaN  

[6475503 rows x 12 columns]

In [108]:
Stocker2.data_ta['0050']

KeyError: '0050'

In [102]:
flaot_cols = ['開盤價', '最高價', '最低價', '收盤價']
int_cols = ['成交股數',	'成交筆數',	'成交金額']
Stocker2.data[flaot_cols] = Stocker2.data[flaot_cols].apply(lambda x: pd.to_numeric(x, errors='coerce'))
Stocker2.data[int_cols] = Stocker2.data[int_cols].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [113]:
Stocker2.data_ta.loc['close',('證券代號')]

KeyError: 'close'

In [114]:
Stocker2.data_ta['close']

證券代號  交易日       
0050  2025-02-19    197.30
      2025-02-20    196.55
      2025-02-21    197.95
      2025-02-24    196.20
      2025-02-25    193.65
                     ...  
9958  NaT           169.00
      NaT           170.50
      NaT           170.50
      NaT           168.50
      NaT           168.50
Name: close, Length: 6475503, dtype: float64

In [115]:
Stocker2.data_ta.groupby(level=0, group_keys=False).apply(
    lambda group: ta.rsi(group["close"])
)

證券代號  交易日       
0050  2025-02-19          NaN
      2025-02-20          NaN
      2025-02-21          NaN
      2025-02-24          NaN
      2025-02-25          NaN
                      ...    
9958  NaT           43.897413
      NaT           51.002698
      NaT           51.002698
      NaT           42.650016
      NaT           42.650016
Length: 7177286, dtype: float64

In [116]:
ta.sma(Stocker2.data_ta['close'], length=20)

證券代號  交易日       
0050  2025-02-19   NaN
      2025-02-20   NaN
      2025-02-21   NaN
      2025-02-24   NaN
      2025-02-25   NaN
                    ..
9958  NaT          NaN
      NaT          NaN
      NaT          NaN
      NaT          NaN
      NaT          NaN
Name: SMA_20, Length: 6475503, dtype: float64